# FBS unexpected behavior on 2025-04-19

Expected behavior: 
using the [fbs_config_minimal_fieldsurvey.py](https://github.com/lsst-ts/ts_config_ocs/blob/tickets/DM-49954/Scheduler/feature_scheduler/maintel/fbs_config_minimal_fieldsurvey.py) configuration, 
the observers would slew to the field to be observed, and take sequences of 30 i-band images.
The field choice is driven by the slew basis functions, with some alt/az masks.

The first sequence of Rubin_SV_216_-17 observations went fine, but for the second sequence (where we have no scheduler snapshot) the FBS ran away to the first field in the list, Carina.


In [190]:
import os
import numpy as np
import pandas as pd
import healpy as hp
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta
from rubin_nights import connections
from lsst.resources import ResourcePath
import pickle
from urllib.parse import urlparse

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler.features import Conditions

from rubin_scheduler.scheduler.model_observatory import ModelObservatory

days_to_seconds = 24*60*60

endpoints = connections.get_clients(tokenfile='/Users/lynnej/.lsst/rsp_prod')

In [15]:
# We need to find a range of times to query. 
# If it's early in the night, this can be the easiest - to just use the dayobs. 

In [16]:
# Set a range of times values to query efd and consdb .. 
day_obs = "2025-04-19"

t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = Time(f"{day_obs}T12:00:00", format='isot', scale='utc') + TimeDelta(18 * 60 * 60, format='sec')

print(f"Querying from {t_start.iso} to {t_end.iso}")

Querying from 2025-04-19 12:00:00.000 to 2025-04-20 06:00:00.000


In [193]:
# Get targets from the EFD 

efd_client = endpoints['efd']

salindex = 1
topic = 'lsst.sal.Scheduler.logevent_target'
fields = '*'
targets = efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
delta_target = (targets.index.values[1:] - targets.index.values[:-1]) / np.timedelta64(1, 's')
targets['delta_target'] = np.concatenate([np.array([0]), delta_target])
#print(fields)
targets['count'] = np.arange(0, len(targets))
tcols = ['delta_target', 'ra', 'decl', 'filter', 'exposureTimes0', 'slewTime', 'skyAngle', 'airmass', 'moonRa', 'skyBrightness', 'seeing', 'note', 'targetName', 'targetId', 'count', 'snapshotUri']
targets[tcols]

,delta_target,ra,decl,filter,exposureTimes0,slewTime,skyAngle,airmass,moonRa,skyBrightness,seeing,note,targetName,targetId,count,snapshotUri
time,,,,,,,,,,,,,,,,
2025-04-19 15:40:21.554662+00:00,0.000000,0.000000,0.000000,,0,0.000000,0.000000,0.000000,0,0.000000,0,Target,Target,0,0,
2025-04-19 18:45:51.276207+00:00,11129.721545,0.000000,0.000000,,0,0.000000,0.000000,0.000000,0,0.000000,0,Target,Target,0,1,
2025-04-19 20:04:46.368740+00:00,4735.092533,0.000000,0.000000,,0,0.000000,0.000000,0.000000,0,0.000000,0,Target,Target,0,2,
2025-04-20 03:54:01.026560+00:00,28154.657820,0.000000,0.000000,,0,0.000000,0.000000,0.000000,0,0.000000,0,Target,Target,0,3,
2025-04-20 04:40:41.230415+00:00,2800.203855,216.199853,-16.632919,i,30,120.000000,231.480180,1.042818,0,20.026947,0,Rubin_SV_216_-17,Rubin_SV_216_-17,138,4,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-04-20 04:40:42.470711+00:00,1.240296,216.244840,-16.702192,i,30,120.000000,228.720192,1.034172,0,20.039312,0,Rubin_SV_216_-17,Rubin_SV_216_-17,139,5,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-04-20 04:41:55.641339+00:00,73.170628,216.265098,-16.695412,i,30,120.000000,225.765367,1.034172,0,20.039312,0,Rubin_SV_216_-17,Rubin_SV_216_-17,140,6,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-04-20 04:43:07.841687+00:00,72.200348,216.204822,-16.852893,i,30,120.000000,222.860600,1.034172,0,20.039312,0,Rubin_SV_216_-17,Rubin_SV_216_-17,141,7,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-04-20 04:45:06.359687+00:00,118.518000,216.003588,-16.735391,i,30,120.000000,219.096906,1.034172,0,20.039312,0,Rubin_SV_216_-17,Rubin_SV_216_-17,142,8,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...


In [73]:
# The targets -- up until the target at RA=300 / Dec = -40 -- were all acquired as visits. 

consdb = endpoints['consdb']
visits = consdb.get_visits('lsstcam', day_obs, day_obs)

vv = visits.query('science_program == "BLOCK-365"')

vcols = ['exposure_name', 'visit_gap', 'exp_midpt_mjd', 'band', 's_ra', 's_dec', 'sky_rotation', 'target_name']
# trim down visits (since we know where the last relevant one was
vv = vv.query('exp_midpt_mjd <= 60785.225564')
print(len(vv))
display(HTML(vv[vcols].to_html()))

29


,exposure_name,visit_gap,exp_midpt_mjd,band,s_ra,s_dec,sky_rotation,target_name
visit_id,,,,,,,,
2025041900328,MC_O_20250419_000328,277.289636,60785.195978,i,216.199853,-16.632919,231.482406,Rubin_SV_216_-17
2025041900329,MC_O_20250419_000329,42.000567,60785.196822,i,216.244840,-16.702192,228.722901,Rubin_SV_216_-17
2025041900330,MC_O_20250419_000330,86.395822,60785.198180,i,216.265098,-16.695412,225.767616,Rubin_SV_216_-17
2025041900331,MC_O_20250419_000331,42.377717,60785.199029,i,216.204822,-16.852893,222.863250,Rubin_SV_216_-17
2025041900332,MC_O_20250419_000332,41.772138,60785.199870,i,216.003588,-16.735391,219.099242,Rubin_SV_216_-17
2025041900333,MC_O_20250419_000333,46.793120,60785.200770,i,216.263461,-16.742987,216.842499,Rubin_SV_216_-17
2025041900334,MC_O_20250419_000334,51.043938,60785.201719,i,216.057250,-16.633716,213.084803,Rubin_SV_216_-17
2025041900335,MC_O_20250419_000335,44.634019,60785.202594,i,215.980636,-16.563157,209.754447,Rubin_SV_216_-17
2025041900336,MC_O_20250419_000336,79.510801,60785.203872,i,216.208581,-16.862855,207.891391,Rubin_SV_216_-17


In [74]:
# The visits track the targets up to the point where the FBS tried to point at the Rubin_SV_300_-4 field at RA=300, Dec=-40
# What was the alt/az of this pointing at that time?  
# (there was some concern that the FBS was trying to point at something that had not yet risen ..)

from rubin_scheduler.utils import approx_ra_dec2_alt_az
from rubin_scheduler.utils import Site

v = visits.loc[2025041900356]

lsst = Site('LSST')
mjd = v.obs_start_mjd
ra= v.s_ra
dec = v.s_dec

alt, az = approx_ra_dec2_alt_az(ra, dec, lsst.latitude, lsst.longitude, mjd)
print(alt, az, az-360)

# So Rubin_SV_300 target was at altitude of 31 degrees -- so it was really low, but not forbidden by the altitude limits available (20 deg).

76.38122208566844 349.4678433511708 -10.532156648829186


In [194]:
# What was the scheduler snapshot looking like? We only have one from the start of the first target
# Snapshots are saved when the FBS is queried for "request_observation" 
# This currently happens for two calls in a row, then the resulting sequences are parsed in sets of twos by the Scheduler queue

# Get the snapshots 

scols = [ 'ra', 'decl', 'filter', 'skyAngle', 'note', 'targetName', 'targetId', 'snapshotUri']
display(HTML(targets[scols].to_html()))


,ra,decl,filter,skyAngle,note,targetName,targetId,snapshotUri
time,,,,,,,,
2025-04-19 15:40:21.554662+00:00,0.000000,0.000000,,0.000000,Target,Target,0,
2025-04-19 18:45:51.276207+00:00,0.000000,0.000000,,0.000000,Target,Target,0,
2025-04-19 20:04:46.368740+00:00,0.000000,0.000000,,0.000000,Target,Target,0,
2025-04-20 03:54:01.026560+00:00,0.000000,0.000000,,0.000000,Target,Target,0,
2025-04-20 04:40:41.230415+00:00,216.199853,-16.632919,i,231.480180,Rubin_SV_216_-17,Rubin_SV_216_-17,138,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/04/19/Scheduler:1_Scheduler:1_2025-04-20T04:41:15.918.p
2025-04-20 04:40:42.470711+00:00,216.244840,-16.702192,i,228.720192,Rubin_SV_216_-17,Rubin_SV_216_-17,139,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/04/19/Scheduler:1_Scheduler:1_2025-04-20T04:41:15.918.p
2025-04-20 04:41:55.641339+00:00,216.265098,-16.695412,i,225.765367,Rubin_SV_216_-17,Rubin_SV_216_-17,140,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/04/19/Scheduler:1_Scheduler:1_2025-04-20T04:41:15.918.p
2025-04-20 04:43:07.841687+00:00,216.204822,-16.852893,i,222.860600,Rubin_SV_216_-17,Rubin_SV_216_-17,141,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/04/19/Scheduler:1_Scheduler:1_2025-04-20T04:41:15.918.p
2025-04-20 04:45:06.359687+00:00,216.003588,-16.735391,i,219.096906,Rubin_SV_216_-17,Rubin_SV_216_-17,142,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/04/19/Scheduler:1_Scheduler:1_2025-04-20T04:41:15.918.p


In [198]:
targetId = 138
t = targets.query('targetId == @targetId')
url = t.snapshotUri.values[0]
print(t.index[0], url)

# EFD records the summit LFA -- if not at the summit, swap.
bucket = "s3://lfa@"
uri = ResourcePath(bucket + urlparse(url).path.lstrip('/'))

# tenantated buckets names have colons and require disable bucket validation
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"

result = uri.read()

#unpickle
sched, conditions = pickle.loads(result)

# Just check that these are the right kind of things 
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)

2025-04-20 04:40:41.230415+00:00 https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/04/19/Scheduler:1_Scheduler:1_2025-04-20T04:41:15.918.p


In [199]:
#sched.survey_lists

In [200]:
# First visit time matches conditions.mjd, as expected.
print('conditions time', conditions.mjd) 
# Check on alt/az limits in the conditions (to see if anything masked)
print('tel limits', np.degrees(conditions.tel_alt_limits), np.degrees(conditions.tel_az_limits))
# And check on current position of the telescope -- expect this to be ECDFS location, as they pointed there
print('tel pos', np.degrees(conditions.tel_ra), np.degrees(conditions.tel_dec))
print(conditions.current_filter, conditions.mounted_filters)

conditions time 60785.1953125
tel limits [40. 84.] [-200.  200.]
tel pos 216.61328830092458 -16.815914308402007
r ['g', 'r', 'i', 'z', 'y']


In [201]:
# just check on surveys and how many observations they know about .. 
# (there's only one tier in this scheduler configuration)
# Might as well check total # of visits against consdb previous visits 
# (since this is start of the day, these should be all from previous nights)
#visit_query = f'''SELECT * FROM cdb_{instrument}.visit1 where day_obs < {day_obs_int} and science_program like '%%BLOCK-365%%' '''
#visits = consdb.query(visit_query)

for s in sched.survey_lists[0]:
    # ok, the list is long .. only do this for the ones with non-zero obs for the survey .. 
    if s.extra_features['ObsRecorded_note'].feature > 0:
        print("survey", s.survey_name, np.degrees(s.ra), np.degrees(s.dec))
        # hmm .. not all previous visits are counted .. but perhaps some of the previous visits were bad?
        print("nvisits", s.extra_features['ObsRecorded'].feature, s.extra_features['ObsRecorded_note'].feature, len(visits.query('target_name == @s.survey_name')))

In [202]:
# What times are we talking about .. 
v = visits.loc[2025041900328]
t = targets.query('targetId == 138')
print(conditions.mjd, v.exp_midpt_mjd, Time(t.index).utc.mjd)
print("first visit - conditions time (seconds)")
print((v.exp_midpt_mjd - conditions.mjd) * days_to_seconds )
print("conditions - target recorded time (seconds)")
print((conditions.mjd - Time(t.index).utc.mjd) * days_to_seconds)

60785.1953125 60785.195977672694 [60785.19492165]
first visit - conditions time (seconds)
57.47092079836875
conditions - target recorded time (seconds)
[33.76958524]


In [203]:
# All right - let's go update the scheduler to the current conditions, and ask for an observation. 
print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
sched.update_conditions(conditions)
print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
# ... so maybe at the start of the night, the conditions pickled in the scheduler match the current conditions. 

# Also, given the times above .. the conditions.mjd is not quite the time the visit is acquired, 
#but it's also not the time the target is recorded .. so let's assume it's the goal visit start time.

60785.1953125 60785.1953125 3.7806150844228474
60785.1953125 60785.1953125 3.7806150844228474


In [204]:
obs = sched.request_observation(conditions.mjd, whole_queue=True)

In [205]:
print(len(obs))
obs[0]['target_name']

30


ObservationArray(['Rubin_SV_216_-17'], dtype='<U40')

In [208]:
obs[0]

ObservationArray([(0, 3.77339928, -0.2902992, 0., 60792.49614583, 30., 'i', 'i_39', 4.0400913, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'Rubin_SV_216_-17', 'Rubin_SV_216_-17', 138, 0., -0.42496264, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'field_survey_science', 'BLOCK-365', 0., 'Rubin_SV_216_-17')],
                 dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('band', '<U40'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('target_id', '<i8'), 

In [209]:
s = sched.survey_lists[0][3]
for bf in s.basis_functions:
    print(bf)
    val = bf(conditions)
    try:
        print(val[s.roi_hpid])
    except:        
        print(val)

0
0.0
0
-0.03413071729098283


In [239]:
# .. I don't know what the second call to the FBS would look like. 
# Let's assume we set conditions tel_ra and tel_dec to the location of the last requested observation

conditions2 = Conditions()
conditions2.set_maintel_info(
        #mjd=visits.loc[2025041900356].obs_end_mjd,
        mjd = conditions.mjd + ((34+8) * 30) / 60 / 60 / 24,
        slewtime=conditions.slewtime /conditions.slewtime,
        current_filter=conditions.current_filter,  # to become current_band
        mounted_filters=conditions.mounted_filters,  # to become mounted_bands
        night=conditions.night,
        skybrightness=conditions.skybrightness,
        fwhm_eff=conditions.fwhm_eff,
        moon_alt=conditions.moon_alt,
        moon_az=conditions.moon_az,
        moon_ra=conditions.moon_ra,
        moon_dec=conditions.moon_dec,
        moon_phase=conditions.moon_phase,
        sun_alt=conditions.sun_alt,
        sun_az=conditions.sun_az,
        sun_ra=conditions.sun_ra,
        sun_dec=conditions.sun_dec,
        tel_ra=conditions.tel_ra,
        tel_dec=conditions.tel_dec,
        tel_alt=conditions.tel_alt,
        tel_az=conditions.tel_az,
        wind_speed=conditions.wind_speed,
        wind_direction=conditions.wind_direction,
        sun_n12_setting=conditions.sun_n12_setting,
        sun_n12_rising=conditions.sun_n12_rising,
        sun_n18_setting=conditions.sun_n18_setting,
        sun_n18_rising=conditions.sun_n18_rising,
        moonrise=conditions.moonrise,
        moonset=conditions.moonset,
        planet_positions=conditions.planet_positions,
        tel_alt_limits=conditions.tel_alt_limits,
        tel_az_limits=conditions.tel_az_limits,
        sky_alt_limits=conditions.sky_alt_limits,
        sky_az_limits=conditions.sky_az_limits,)
conditions2.sunrise = conditions.sunrise
conditions2.sunset = conditions.sunset
# the last observation doesn't have a time ... 
# the target times are tied to queue operations, and this time wouldn't be available when the next target is requested
# what happens if we just use the conditions again at the same time? 

print(conditions2.mjd)

sched.update_conditions(conditions2)
obs2 = sched.request_observation(conditions2.mjd, whole_queue=True)
obs2



60785.20989583333


/Users/lynnej/lsst_repos/rubin_scheduler/rubin_scheduler/scheduler/utils/utils.py:179: UserWarning: IntRounded being used with a potentially too-small scale factor.
  warnings.warn("IntRounded being used with a potentially too-small scale factor.")


[ObservationArray([(0, 2.80976694, -1.03939764, 0., 60786.89472917, 30., 'i', 'i_39', 1.63921989, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'Carina', 'Carina', 258, 0., -0.35216762, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'field_survey_science', 'BLOCK-365', 0., 'Carina')],
                  dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('band', '<U40'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('target_id', '<i8'), ('lmst', '<f8'), ('rotTelP

In [212]:
s = sched.survey_lists[0][3]
for bf in s.basis_functions:
    print(bf)
    val = bf(conditions2)
    try:
        print(val[s.roi_hpid])
    except:        
        print(val)
all_rewards_this_survey = s.calc_reward_function(conditions2)
rewards = (np.nan if np.all(np.isnan(all_rewards_this_survey)) else np.nanmax(all_rewards_this_survey))
rewards
result = s.generate_observations(conditions2)
result[0]

0
0.0
0
-0.03413071729098283


np.void((0, 3.7724841984064508, -0.290253948294922, 0.0, 60792.52589314726, 30.0, 'i', '', 3.879386408178891, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'Rubin_SV_216_-17', 'Rubin_SV_216_-17', 0, 0.0, -0.9076573672002577, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'field_survey_science', 'BLOCK-365', 0.0, ''), dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('band', '<U40'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('tar

In [213]:
# ah .. once we pull the observations out of the queue, 
sched.update_conditions(conditions2)
sched._fill_queue()
winner = sched.survey_index
ss = sched.survey_lists[winner[0]][winner[1]]
ss.survey_name, winner
# Okay, we've reproduced the "runaway" behavior -- let's see why this survey won

('Rubin_SV_216_-17', [0, np.int64(3)])

In [232]:
mjd = np.arange(conditions.mjd, conditions.mjd + ((34+8) * 30) / 60 / 60 / 24, 5/60/60/24)
ra= np.degrees(s.ra)
dec = np.degrees(s.dec)

alt, az = approx_ra_dec2_alt_az(ra, dec, lsst.latitude, lsst.longitude, mjd)
#print(alt, az, az-360)
print(alt)

[47.94355264 47.93405315 47.92455192 47.91504896 47.90554427 47.89603785
 47.8865297  47.87701983 47.86750823 47.85799491 47.84847988 47.83896312
 47.82944465 47.81992446 47.81040257 47.80087896 47.79135365 47.78182663
 47.7722979  47.76276748 47.75323535 47.74370153 47.73416601 47.72462879
 47.71508988 47.70554928 47.696007   47.68646303 47.67691737 47.66737003
 47.65782101 47.64827031 47.63871793 47.62916388 47.61960815 47.61005076
 47.60049169 47.59093096 47.58136856 47.5718045  47.56223877 47.55267139
 47.54310235 47.53353165 47.5239593  47.5143853  47.50480965 47.49523235
 47.4856534  47.47607281 47.46649057 47.4569067  47.44732119 47.43773404
 47.42814525 47.41855483 47.40896278 47.39936911 47.3897738  47.38017687
 47.37057831 47.36097814 47.35137634 47.34177293 47.3321679  47.32256126
 47.312953   47.30334313 47.29373166 47.28411858 47.27450389 47.26488761
 47.25526972 47.24565023 47.23602914 47.22640646 47.21678219 47.20715632
 47.19752887 47.18789983 47.1782692  47.16863699 47

In [221]:
s = sched.survey_lists[0][0]
for bf in s.basis_functions:
    print(bf)
    val = bf(conditions2)
    try:
        print(val[s.roi_hpid])
    except:        
        print(val)

0
nan
0
-0.8831825895099432


In [217]:
# Look at all of the rewards 
reward_df = sched.make_reward_df(conditions=conditions, accum=True)
summary_df = reward_df.reset_index()
summary_df[['survey_index', 'survey_label', 'survey_reward']].groupby('survey_index').max()

,survey_label,survey_reward
survey_index,,
0,Carina,-0.220796
1,Trifid-Lagoon,NaN
2,Prawn,-0.106056
3,Rubin_SV_216_-17,-0.008533
4,Rubin_SV_225_-19,-0.029685
5,COSMOS,NaN
6,ELAIS_S1,NaN
7,XMM_LSS,NaN
8,ECDFS,NaN


In [215]:
# It's odd that they're all 0 of nan ..
# Let's look at the reward basis value functions for the winner
reward_df.loc[winner[0], winner[1]]

basis_function  \
list_index survey_index                           
0          3                        NotTwilight   
           3                    AltAzShadowMask   
           3             AltAzShadowTimeLimited   
           3                           Slewtime   

                                        basis_function_class  feasible  \
list_index survey_index                                                  
0          3                        NotTwilightBasisFunction      True   
           3                    AltAzShadowMaskBasisFunction      True   
           3             AltAzShadowTimeLimitedBasisFunction      True   
           3                           SlewtimeBasisFunction      True   

                         max_basis_reward  basis_area  basis_weight  \
list_index survey_index                                               
0          3                     0.000000    3.357175          0.25   
           3                     0.000000    3.357175          0.25   
           3                     0.000000    3.357175          0.25   
           3                    -0.034131    3.357175          0.25   

                         max_accum_reward  accum_area tier_label  \
list_index survey_index                                            
0          3                     0.000000    3.357175     tier 0   
           3                     0.000000    3.357175     tier 0   
           3                     0.000000    3.357175     tier 0   
           3                    -0.008533    3.357175     tier 0   

                             survey_label survey_class  survey_reward  
list_index survey_index                                                
0          3             Rubin_SV_216_-17  FieldSurvey      -0.008533  
           3             Rubin_SV_216_-17  FieldSurvey      -0.008533  
           3             Rubin_SV_216_-17  FieldSurvey      -0.008533  
           3             Rubin_SV_216_-17  FieldSurvey      -0.008533

In [238]:
t_start = Time("2025-04-20 04:40:41.230415", format='iso', scale='utc')
nominal_sequence = (32.5+5) * 30 / 60/60/24
t_end = t_start + TimeDelta(nominal_sequence, format='jd')

topic = 'lsst.sal.ScriptQueue.logevent_message'
efd_client.select_time_series(topic, '*', t_start, t_end)

""
